In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
import ast
import matplotlib.pyplot as plt
import cv2
from glob import glob
import tensorflow as tf



In [ ]:
fnames = glob('../input/train_simplified/*.csv')
cnames = ['countrycode', 'drawing', 'key_id', 'recognized', 'timestamp', 'word','class_id']
drawlist = []
files_number = 340
for i in range(files_number):
    f = fnames[i]
    first = pd.read_csv(f)
    first = first[first.recognized==True]
    first[6] = i
    drawlist.append(first)
draw_df = pd.DataFrame(np.concatenate(drawlist),columns = cnames)
train_size = len(draw_df)

In [ ]:
def draw2img(drawing, shape =(32,32), lw = 4):
    fig, ax = plt.subplots()
    for x,y in drawing:
        ax.plot(x, y,'g',  marker='.', linewidth = lw)
    ax.axis('off')
    fig.canvas.draw()    
    X = np.array(fig.canvas.renderer._renderer)
    plt.close(fig)
    temp = (cv2.resize(X, shape) / 255.)[::-1]
    return temp[:,:,1] 

In [ ]:
model = tf.keras.models.Sequential([
  tf.keras.layers.Flatten(),
  tf.keras.layers.Dense(512, activation=tf.nn.relu),
  tf.keras.layers.Dropout(0.2),
  tf.keras.layers.Dense(340, activation=tf.nn.softmax)
])
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

model.fit(x_train, y_train, epochs=train_size)

In [ ]:
tests = pd.read_csv('../input/test_simplified.csv')
test_drawings = [ast.literal_eval(pts) for pts in tests[0:len(tests)]['drawing'].values] 
test_len = len(test_drawings)
test_images = []
for d in test_drawings:
        test_image = np.array(draw2img(d,(32,32),4)).reshape((32, 32))
        test_images.append(test_image)
x_test = np.array(test_images)

In [ ]:
p = model.predict_classes(x_test[0:test_len])
classfiles = os.listdir('../input/train_simplified/')
numstonames = {i: v[:-4].replace(" ", "_") for i, v in enumerate(classfiles)}
sub = pd.read_csv('../input/sample_submission.csv', index_col=['key_id'])
coutnames = ['words']
out = pd.DataFrame(p,columns = coutnames)
out = out.replace(numstonames)
sub['word'] = out.words.values
sub.to_csv('output.csv')